# Application of Parametric NN to data

In [1]:
from ROOT import TMVA, TFile, TTree, TH1F, THStack, TLorentzVector, TPad, TCanvas, TLegend, TH2D
import numpy as np
from array import array
from numpy import sqrt as Sqrt
import pandas as pd
from os import chdir, listdir
from tensorflow.keras.models import Sequential, load_model
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

Welcome to JupyROOT 6.22/06


In [2]:
csvFolder = './merged/'
weightFolder = './saved_models/'
bkg_path = './merged/background/' 
sgn_path = './merged/signal/'
data_path = './merged/data/'

In [3]:
cat = '' # 'cat2'

In [4]:
    if cat == 'cat2':
        variables = [
            ("dimuon_deltar", "h_dimuon_deltar", "Dimuon Delta R; R; events", 60, 0, 6),                                                                                                #Define discrimination variables
            ("dimuon_deltaphi", "h_dimuon_deltaphi", "Dimuon Delta phi; phi; events", 35, 0, 3.5),
            ("dimuon_deltaeta","h_dimuon_deltaeta", "Dimuon Delta eta;eta;events", 25, 0, 5),
            ("met_pt","h_met_pt", "E_{T}^{miss};E_{T};events",50, 0, 500),
            ("ljet_1_pt","h_ljet_1_pt", "p_{T} first ljet;p_{T};ljet", 50, 0, 500),
            ("ljet_1_eta","h_ljet_1_eta", "eta first light jet;eta;ljet", 30, -5, 5),
            ("ljet_n","h_ljet_n", "Number of light jet;n ljet;events",10, 0, 10),
            ]
    else:
        variables = [
                    ("dimuon_deltar", "h_dimuon_deltar", "Dimuon Delta R; R; events", 60, 0, 6),                                                                                                #Define discrimination variables
                    ("dimuon_deltaphi", "h_dimuon_deltaphi", "Dimuon Delta phi; phi; events", 35, 0, 3.5),
                    ("dimuon_deltaeta","h_dimuon_deltaeta", "Dimuon Delta eta;eta;events", 25, 0, 5),
                    ("met_pt","h_met_pt", "E_{T}^{miss};E_{T};events",50, 0, 500),
                    ("deltar_bjet1_dimuon","h_deltar_bjet1_dimuon", "Delta R first bjet-dimuon; Delta R; events", 100, 0, 10),
                    ("deltapt_bjet1_dimuon","h_deltapt_bjet1_dimuon", "Delta p_{T} first bjet-dimuon", 100, 0, 300),
                    ("deltaeta_bjet1_dimuon","h_deltaeta_bjet1_dimuon", "Delta eta first bjet-dimuon;Delta eta;events", 40, 0, 8),
                    ("bjet_1_pt","h_bjet_1_pt", "p_{T} of first bjet;p_{T};bjet", 50, 0, 500),
                    ("bjet_1_eta","h_bjet_1_eta", "eta first bjet;eta;bjet", 30, -5, 5),
                    ("ljet_1_pt","h_ljet_1_pt", "p_{T} first ljet;p_{T};ljet", 50, 0, 500),
                    ("ljet_1_eta","h_ljet_1_eta", "eta first light jet;eta;ljet", 30, -5, 5),
                    ("bjet_n","h_bjet_n", "Number of bjet;n bjet; events",10, 0, 10),
                    ("ljet_n","h_ljet_n", "Number of light jet;n ljet;events",10, 0, 10),
                    ]

In [5]:
nn_features = ["mA",
            "dimuon_deltar",
            "dimuon_deltaphi",
            "dimuon_deltaeta",
            "met_pt",
            "deltar_bjet1_dimuon",
            "deltapt_bjet1_dimuon",
            "deltaeta_bjet1_dimuon",
            "bjet_1_pt",
            "bjet_1_eta",
            "ljet_1_pt",
            "ljet_1_eta",
            "bjet_n",
            "ljet_n"]

In [6]:
lumi = 35858.5
massA = 700
massA = int(massA)
range_up = massA + massA*0.1
range_down = massA-massA*0.1

In [7]:
model = load_model(weightFolder + 'signal_weighted_model_a.h5')
best_cut = 0.65
batch_size = 10000

2021-02-17 17:47:14.297909: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-17 17:47:14.298343: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
sgn_data = [
    (sgn_path + s, 1) for s in listdir(sgn_path)
]

bkg_data = [
    (sgn_path + "bbH_MA"+str(massA)+"_tanb20_2016.csv", 7.98915815353/100., 1),
    (sgn_path + "bbA_MA"+str(massA)+"_tanb20_2016.csv", 0.923222780228/100., 1),
    
    (bkg_path + "diboson_WZTo2L2Q.csv", 4.409/26517272., 5),
    (bkg_path + "diboson_WWTo2L2Nu.csv", 12.46/1999000., 5),
    (bkg_path + "diboson_WZTo3LNu.csv", 2.113/11928707., 5),
    (bkg_path + "diboson_ZZTo2L2Nu.csv", 0.564/57586850., 5),
    (bkg_path + "diboson_ZZTo2L2Q.csv", 3.22/15462693., 5),
                    
    (bkg_path + "ST_schannel.csv", 6.35/1000000., 4),
    (bkg_path + "ST_tchannel_antitop.csv", 80.95/38811017., 4),
    (bkg_path + "ST_tchannel_top.csv", 136.02/67105876., 4),
    (bkg_path + "ST_tW_antitop.csv", 35.85/11408144., 4),
    (bkg_path + "ST_tW_top.csv", 35.85/11397473., 4),
                    
    (bkg_path +"ttbar.csv", 85.656/30836035., 2),
                    
    (bkg_path +"DY.csv", 5765./120777245., 3)
    
]

groups = [2, 7, 12, 13, 14]
real_data = [
    (data_path + "Run2016B.csv", 1),
    (data_path + "Run2016C.csv", 1),
    (data_path + "Run2016D.csv", 1),
    (data_path + "Run2016E.csv", 1),
    (data_path + "Run2016F.csv", 1),
    (data_path + "Run2016G.csv", 1),
    (data_path + "Run2016H.csv", 1)
    ]

In [9]:
plot = {}
for f in bkg_data:
    plot[f[0]] = {}
s_files = {}
s_trees = {}
b_files = {}
b_trees = {}
d_files = {}
d_trees = {}
s_plots = {}
b_plots = {}
d_plots = {}
#PU_Weight 	= array('f', [-999])

In [10]:
for v in variables:
    s_plots[v[0]] = TH1F('s_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    b_plots[v[0]] = TH1F('b_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    d_plots[v[0]] = TH1F('d_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))

In [11]:
s_inv_mass = TH1F('s_invariant_mass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)
b_inv_mass = TH1F('b_invariant_mass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)
d_inv_mass = TH1F('d_invariant_mass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)
pre_s_inv_mass = TH1F('pre_s_invariant_mass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)
pre_b_inv_mass = TH1F('pre_b_invariant_mass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)
pre_d_inv_mass = TH1F('pre_d_invariant_mass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)

### Pre-Analysis Plots

In [ ]:
print('\n==>Starting pre analysis procedure...\n')

out_ = TFile.Open('preAnalisis_plots_{}.root'.format(cat), 'RECREATE')  
bkg_histo = {}
data_histo = {}
tmpsig0 = {}
tmpbkg0 = {}
tmpbkg1 = {}
tmpbkg2 = {}
tmpbkg3 = {}
tmpbkg4 = {}
stacked_histo = {}
legend_stack = {}

tmp = {}

In [ ]:
for v in variables:
    bkg_histo[v[0]]  = TH1F('_p_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    data_histo[v[0]] = TH1F('_d_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpsig0[v[0]] = TH1F('tmp0_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpbkg0[v[0]] = TH1F('tmp1_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpbkg1[v[0]] = TH1F('tmp2_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpbkg2[v[0]] = TH1F('tmp3_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpbkg3[v[0]] = TH1F('tmp4_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpbkg4[v[0]] = TH1F('tmp5_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    stacked_histo[v[0]] = THStack(v[1], v[2])
    legend_stack[v[0]] = TLegend(0.71,0.7,0.88,0.89)
    
for f in bkg_data:
    tmp[f[0]] = {}
s_tmp = {}

In [ ]:
for s in sgn_data:
    s_df = pd.read_csv(s[0])
    s_tmp[s[0]] = TH1F(s[0] + 'invMass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)
    n = len(s_df)
    print('\nSignal tree {} with {} events'.format(s[0], n))
    i=0
    for index, row in s_df.iterrows():
        if i%1000 == 0:
            print('Processing event {}'.format(i))
            s_tmp[s[0]].Fill(row["dimuon_M"], row["PU_Weight"])
        i = i+1
                
for s in sgn_data:
    scale = s_tmp[s[0]].GetEntries()/s_tmp[s[0]].Integral() if s_tmp[s[0]].Integral() > 0 else 1
    pre_s_inv_mass.Add(s_tmp[s[0]], scale * s[1] * lumi)

In [ ]:
for b in bkg_data:
    b_df = pd.read_csv(b[0])
    n = len(b_df)
    print('Background {} with {} events'.format(b[0], n))
    for v in variables:
        tmp[b[0]][v[0]] = TH1F(b[0] + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmp[b[0]]['dimuonMass'] = TH1F(b[0]+'b_invariant_mass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)
    i=0
    for index, row in b_df.iterrows():
        if i%10000 == 0:
            print('\nProcessing event {}'.format(i))
            
        scale =  row["PU_Weight"]
        M = row["dimuon_M"]
        dimuon_deltar = row["dimuon_deltar"]
        dimuon_deltaphi = row["dimuon_deltaphi"]
        dimuon_deltaeta = row["dimuon_deltaeta"]
        met_pt = row["met_pt"]
        ljet_n = row["ljet_n"]
            
        tmp[b[0]]["dimuonMass"].Fill(M, scale)
        tmp[b[0]]["dimuon_deltar"].Fill(dimuon_deltar, scale)
        tmp[b[0]]["dimuon_deltaphi"].Fill(dimuon_deltaphi, scale)
        tmp[b[0]]["dimuon_deltaeta"].Fill(dimuon_deltaeta, scale)
        tmp[b[0]]["met_pt"].Fill(met_pt, scale)
        tmp[b[0]]["ljet_n"].Fill(ljet_n, scale)
            
        if cat != 'cat2':
            bjet_n = row["bjet_n"]
            bjet_1_pt = row["bjet_1_pt"]
            bjet_1_eta = row["bjet_1_eta"]
            deltar_bjet1_dimuon = row["deltar_bjet1_dimuon"]
            deltapt_bjet1_dimuon = row["deltapt_bjet1_dimuon"]
            deltaeta_bjet1_dimuon = row["deltaeta_bjet1_dimuon"]
            ljet_1_pt = row["ljet_1_pt"]
            ljet_1_eta = row["ljet_1_eta"]

            tmp[b[0]]["bjet_n"].Fill(bjet_n, scale)
            if bjet_1_pt>0:
                tmp[b[0]]["bjet_1_pt"].Fill(bjet_1_pt, scale)
                tmp[b[0]]["bjet_1_eta"].Fill(bjet_1_eta, scale)
                if deltar_bjet1_dimuon > 0:
                    tmp[b[0]]["deltar_bjet1_dimuon"].Fill(deltar_bjet1_dimuon, scale)
                if deltapt_bjet1_dimuon > 0:
                    tmp[b[0]]["deltapt_bjet1_dimuon"].Fill(deltapt_bjet1_dimuon, scale)
                if deltaeta_bjet1_dimuon > 0:
                    tmp[b[0]]["deltaeta_bjet1_dimuon"].Fill(deltaeta_bjet1_dimuon, scale)
        if ljet_1_pt > 0:
            tmp[b[0]]["ljet_1_pt"].Fill(ljet_1_pt, scale)
            tmp[b[0]]["ljet_1_eta"].Fill(ljet_1_eta, scale)
        i = i+1
    
i = 1
for b in bkg_data:
    leg_bool=False
    for group in groups:
        if i==group:
            leg_bool=True
                
    scale = tmp[b[0]]['dimuonMass'].GetEntries()/tmp[b[0]]['dimuonMass'].Integral() 
    tmp[b[0]]['dimuonMass'].Scale(lumi*b[1]*scale)
    pre_b_inv_mass.Add(tmp[b[0]]['dimuonMass'], 1)
    for v in variables:
        _p = tmp[b[0]][v[0]]
        scale =  _p.GetEntries()/_p.Integral()
        _p.SetFillColor(b[2])
        if b[2] == 1:
            tmpsig0[v[0]].Add(_p, lumi* b[1] *scale)
            if leg_bool == True:
                tmpsig0[v[0]].SetLineColor(1)
                tmpsig0[v[0]].SetLineStyle(1)
                tmpsig0[v[0]].SetLineWidth(3)
                tmpsig0[v[0]].Scale(0.001)
                legend_stack[v[0]].AddEntry(_p,"A m_{A}="+ str(massA) + "GeV x 100","le")
        if b[2] == 3:
            tmpbkg0[v[0]].Add(_p, lumi* b[1] *scale)
            if leg_bool==True:
                tmpbkg0[v[0]].SetLineColor(1)
                tmpbkg0[v[0]].SetFillColor(b[2])
                stacked_histo[v[0]].Add(tmpbkg0[v[0]])
                legend_stack[v[0]].AddEntry(_p,"Drell-Yan","F")
        if b[2] == 2:
            tmpbkg1[v[0]].Add(_p, lumi* b[1] *scale)
            if leg_bool==True:
                tmpbkg1[v[0]].SetLineColor(1)
                tmpbkg1[v[0]].SetFillColor(b[2])
                stacked_histo[v[0]].Add(tmpbkg1[v[0]])
                legend_stack[v[0]].AddEntry(_p,"t#bar{t}","F")
        if b[2] == 4:
            tmpbkg2[v[0]].Add(_p, lumi* b[1] *scale)
            if leg_bool==True:
                tmpbkg2[v[0]].SetLineColor(1)
                tmpbkg2[v[0]].SetFillColor(b[2])
                stacked_histo[v[0]].Add(tmpbkg2[v[0]])
                legend_stack[v[0]].AddEntry(_p,"Single t","F")
        if b[2] == 5:
            tmpbkg3[v[0]].Add(_p, lumi* b[1] *scale)
            if leg_bool==True:
                tmpbkg3[v[0]].SetLineColor(1)
                tmpbkg3[v[0]].SetFillColor(b[2])
                stacked_histo[v[0]].Add(tmpbkg3[v[0]]) 
                legend_stack[v[0]].AddEntry(_p,"W^{+}W^{-}, W^{#pm}Z, ZZ","F")
        if b[2]!=1:
            bkg_histo[v[0]].Add(_p, lumi* b[1] *scale)
    i = i+1
    out_.WriteTObject(tmp[b[0]]['dimuonMass'])

In [ ]:
for d in real_data:
    d_df = pd.read_csv(d[0])
    n = len(d_df)
    print('Data {} with {} events'.format(d[0], n))
    i=0
    for index, row in d_df.iterrows():
        if i%10000 == 0:
            print('\nProcessing event {}'.format(i))
        M = row["dimuon_M"]
        dimuon_deltar = row["dimuon_deltar"]
        dimuon_deltaphi = row["dimuon_deltaphi"]
        dimuon_deltaeta = row["dimuon_deltaeta"]
        met_pt = row["met_pt"]
        ljet_n = row["ljet_n"]
        scale = row["PU_Weight"] * d[1]
        
        pre_d_inv_mass.Fill(M, scale)
        data_histo["dimuon_deltar"].Fill(dimuon_deltar, scale)
        data_histo["dimuon_deltaphi"].Fill(dimuon_deltaphi, scale)
        data_histo["dimuon_deltaeta"].Fill(dimuon_deltaeta, scale)
        data_histo["met_pt"].Fill(met_pt, scale)
        data_histo["ljet_n"].Fill(ljet_n, scale)
        
        if cat != 'cat2':
            bjet_n = row["bjet_n"]
            bjet_1_pt = row["bjet_1_pt"]
            bjet_1_eta = row["bjet_1_eta"]
            deltar_bjet1_dimuon = row["deltar_bjet1_dimuon"]
            deltapt_bjet1_dimuon = row["deltapt_bjet1_dimuon"]
            deltaeta_bjet1_dimuon = row["deltaeta_bjet1_dimuon"]
            ljet_1_pt = row["ljet_1_pt"]
            ljet_1_eta = row["ljet_1_eta"]
            
            data_histo["bjet_n"].Fill(bjet_n, scale)
            if bjet_1_pt > 0:
                data_histo["bjet_1_pt"].Fill(bjet_1_pt, scale)
                data_histo["bjet_1_eta"].Fill(bjet_1_eta, scale)
                if deltar_bjet1_dimuon > 0:
                    data_histo["deltar_bjet1_dimuon"].Fill(deltar_bjet1_dimuon, scale)
                if deltapt_bjet1_dimuon > 0:
                    data_histo["deltapt_bjet1_dimuon"].Fill(deltapt_bjet1_dimuon, scale)
                if deltaeta_bjet1_dimuon > 0:
                    data_histo["deltaeta_bjet1_dimuon"].Fill(deltaeta_bjet1_dimuon, scale)

        if ljet_1_pt > 0:
            data_histo["ljet_1_pt"].Fill(ljet_1_pt, scale)
            data_histo["ljet_1_eta"].Fill(ljet_1_eta, scale)
        i = i+1

In [ ]:
K = 1
padA = {}
padB = {}
Ratios = {}
Canvas = {}
Legend = {}

for v in variables:
    Canvas[v[0]] = TCanvas('pre_' + v[0], v[0])

    padA[v[0]] = TPad("padA", "padA", 0, 0.3, 1, 1.0)
    padA[v[0]].SetBottomMargin(0)
    padA[v[0]].SetGridx()
    padA[v[0]].SetGridy()
    padA[v[0]].SetLogy(1)
    padA[v[0]].Draw()
    padA[v[0]].cd()

    bkg_histo[v[0]].SetStats(0)
    stacked_histo[v[0]].Draw("hist F")
    tmpsig0[v[0]].Draw("same")
    #bkg_histo[v[0]].SetFillColor(4)
    #bkg_histo[v[0]].Draw("hist F")
    data_histo[v[0]].SetMarkerStyle(8)
    data_histo[v[0]].Draw("SAME E")

    #Legend[v[0]] = TLegend(0.1,0.7,0.48,0.9)
    #Legend[v[0]].SetHeader("legend", "C")
    #Legend[v[0]].AddEntry(bkg_histo[v[0]], "Monte Carlo")
    #Legend[v[0]].AddEntry(data_histo[v[0]], "Data")
    #Legend[v[0]].Draw()
    legend_stack[v[0]].AddEntry(data_histo[v[0]], "Data")
    legend_stack[v[0]].SetBorderSize(0)
    legend_stack[v[0]].SetLineWidth(0)
    legend_stack[v[0]].SetFillColor(0)
    legend_stack[v[0]].Draw()
        
    Canvas[v[0]].cd()
    padB[v[0]] = TPad("pad2", "pad2", 0, 0.05, 1, 0.3)
    padB[v[0]].SetTopMargin(0.2)
    padB[v[0]].SetGridx()
    padB[v[0]].SetGridy()
    padB[v[0]].Draw()
    padB[v[0]].cd()

    Ratios[v[0]] = data_histo[v[0]].Clone("ratio")
    Ratios[v[0]].GetYaxis().SetTitle("Data/MC")
    Ratios[v[0]].SetLineColor(1)
    Ratios[v[0]].SetMinimum(0)
    Ratios[v[0]].SetMaximum(2)
    Ratios[v[0]].Sumw2()
    Ratios[v[0]].SetStats(0)
    Ratios[v[0]].Divide(bkg_histo[v[0]])
    Ratios[v[0]].SetMarkerStyle(8)
    Ratios[v[0]].Draw("ep")
    out_.WriteTObject(Canvas[v[0]])


Canvas['invMass'] = TCanvas('pre_invMass', 'invMass')

padA['invMass'] = TPad("padA", "padA", 0, 0.3, 1, 1.0)
padA['invMass'].SetBottomMargin(0)
padA['invMass'].SetGridx()
padA['invMass'].SetGridy()
padA['invMass'].SetLogy(1)
padA['invMass'].Draw()
padA['invMass'].cd()

pre_b_inv_mass.SetStats(0)
pre_b_inv_mass.SetFillColor(4)
pre_b_inv_mass.Draw("hist F")
pre_d_inv_mass.SetMarkerStyle(8)
pre_d_inv_mass.Draw("SAME E")
pre_s_inv_mass.SetMarkerStyle(34)
pre_s_inv_mass.Draw("SAME E")

Legend['invMass'] = TLegend(0.1,0.7,0.48,0.9)
Legend['invMass'].SetHeader("legend", "C")
Legend['invMass'].AddEntry(pre_b_inv_mass, "Monte Carlo")
Legend['invMass'].AddEntry(pre_d_inv_mass, "Data")
Legend['invMass'].AddEntry(pre_s_inv_mass, "bb Signal")
Legend['invMass'].Draw()

Canvas['invMass'].cd()
padB['invMass'] = TPad("padB", "padB", 0, 0.05, 1, 0.3)
padB['invMass'].SetTopMargin(0.2)
padB['invMass'].SetGridx()
padB['invMass'].SetGridy()
padB['invMass'].Draw()
padB['invMass'].cd()

Ratios['invMass'] = pre_d_inv_mass.Clone("ratio")
Ratios['invMass'].GetYaxis().SetTitle("Data/MC")
Ratios['invMass'].SetLineColor(1)
Ratios['invMass'].SetMinimum(0)
Ratios['invMass'].SetMaximum(2)
Ratios['invMass'].Sumw2()
Ratios['invMass'].SetStats(0)
Ratios['invMass'].Divide(pre_b_inv_mass)
Ratios['invMass'].SetMarkerStyle(8)
Ratios['invMass'].Draw("ep")
out_.WriteTObject(Canvas['invMass'])

out_.WriteTObject(pre_s_inv_mass)
out_.WriteTObject(pre_b_inv_mass)
out_.Close()

#NORMALIZZAZIONE = pre_b_inv_mass.Integral()/pre_s_inv_mass.Integral() #MODIFICA

NORMALIZZAZIONE = float(K) #pre_b_inv_mass.Integral(pre_b_inv_mass.FindBin(range_down), pre_b_inv_mass.FindBin(range_up))/pre_s_inv_mass.Integral(pre_s_inv_mass.FindBin(range_down), pre_s_inv_mass.FindBin(range_up))

pre_s_inv_mass.Scale(NORMALIZZAZIONE)
pre_Ns = pre_s_inv_mass.Integral(pre_s_inv_mass.FindBin(range_down), pre_s_inv_mass.FindBin(range_up))
pre_Nb = pre_b_inv_mass.Integral(pre_b_inv_mass.FindBin(range_down), pre_b_inv_mass.FindBin(range_up))

pre_significance_MC_signal = (pre_Ns/Sqrt(pre_Ns+pre_Nb), pre_Ns/Sqrt(pre_Nb))
#pre_significance_data_signal = (pre_Ns/Sqrt(pre_Ns+pre_Nd), pre_Ns/Sqrt(pre_Nd))

stringC = "\nLa massa vale :{}. Ns vale: {}. Nb vale: {}\n".format(massA, pre_Ns, pre_Nb)
stringD = "range_down {} corrisponde a {} e range_up {} a {}".format(range_down, pre_s_inv_mass.FindBin(range_down), range_up, pre_s_inv_mass.FindBin(range_up))
stringA ='\n Significance MC at {}+-10% GeV con taglio a {} is(s/sqrt(s+b)): {}\n'.format(massA, best_cut, pre_significance_MC_signal[0])
stringB = '\nSignificance MC at {}+-10% GeV con taglio a {} is(s/sqrt(b)): {}\n'.format(massA, best_cut, pre_significance_MC_signal[1])
print(stringC)
print(stringD)
print(stringA)
print(stringB)


with open('pre_significancePNN{}.txt'.format(cat), 'w+') as f:
    f.write(stringC)
    f.write(stringD)
    f.write(stringA)
    f.write(stringB)
    f.close()


print("\n==>The pre-Analisis plots are completed...\n")


### Post-Analysis Plots

In [12]:
################################################ ANALYSIS ############################################################

#reader.BookMVA('BDT_{}'.format(cat), weights_path + 'TMVAClassification_BDT_{}.weights.xml'.format(cat))
#reader.BookMVA('PyKeras_deep_{}'.format(cat), weights_path + 'TMVAClassification_PyKeras_deep_{}.weights.xml'.format(cat))

bkg_plot = TH2D("b_discriminante_Vs_massa", "discriminante Vs massa;discriminante;massa",40, -1, 1, 700, 100, 1500)
sgn_plot = TH2D("s_discriminante_Vs_massa", "discriminante Vs massa;discriminante;massa",40, -1, 1, 700, 100, 1500)

#input_file = TFile.Open(input_file_name)
output_file = TFile("output_plots.root", 'RECREATE')

#print("\n--- TMVAClassificationApp    : Using input file: " + input_file_name)

print('\nBest cut is at {}\n'.format(best_cut))

tmpsig0 = {}
tmpbkg0 = {}
tmpbkg1 = {}
tmpbkg2 = {}
tmpbkg3 = {}
tmpbkg4 = {}
stacked_histo = {}
legend_stack = {}

for v in variables:
    tmpsig0[v[0]] = TH1F('tmppost0_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpbkg0[v[0]] = TH1F('tmppost1_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpbkg1[v[0]] = TH1F('tmppost2_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpbkg2[v[0]] = TH1F('tmppost3_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpbkg3[v[0]] = TH1F('tmppost4_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    tmpbkg4[v[0]] = TH1F('tmppost5_' + v[1], v[2], int(v[3]), float(v[4]), float(v[5]))
    stacked_histo[v[0]] = THStack(v[1], v[2])
    legend_stack[v[0]] = TLegend(0.71,0.7,0.88,0.89)


Best cut is at 0.65



In [15]:
print ('\nProcessing signals...\n')
sgn_tmp = {}

#Analysis on signal, sum with just PU_Weight as scaling factor
for s in sgn_data:
    s_df = pd.read_csv(s[0])
    n = len(s_df)
    print('Signal {} with {} events'.format(s[0], n))
    try:
        features = s_df[s_df.columns & nn_features].to_numpy()
        features = scaler.fit_transform(features)
        val = model.predict(features, batch_size = batch_size)
        sgn_tmp[s[0]] = TH1F(s[0] + 'dimuonInvMass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)
        i=0
        for index, row in s_df.iterrows():
            if i%10000 == 0:
                print('\nProcessing event {}'.format(i))
            if cat != 'cat2':
                bjet_n = row["bjet_n"]
            ljet_n = row["ljet_n"]
            M = row["dimuon_M"]
            PU_Weight = row["PU_Weight"]
            sgn_plot.Fill(val[i], M, PU_Weight)
            if val[i] > best_cut:
                sgn_tmp[s[0]].Fill(M, PU_Weight)
            i=i+1
    except ValueError:
        sgn_tmp[s[0]] = TH1F(s[0] + 'dimuonInvMass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)
        continue
print("Here we are")           
for s in sgn_data:
    scale = sgn_tmp[s[0]].GetEntries()/sgn_tmp[s[0]].Integral() if ( sgn_tmp[s[0]].Integral()> 0) else 1 
    s_inv_mass.Add(sgn_tmp[s[0]], scale * s[1] * lumi)


Processing signals...

Signal ./merged/signal/bbA_MA275_tanb10_2016.csv with 2053 events

Processing event 0
Signal ./merged/signal/bbA_MA170_tanb5_2016.csv with 1938 events

Processing event 0
Signal ./merged/signal/bbA_MA200_tanb25_2016.csv with 1979 events

Processing event 0
Signal ./merged/signal/bbH_MA160_tanb35_2016.csv with 0 events
Signal ./merged/signal/bbH_MA150_tanb30_2016.csv with 1836 events

Processing event 0
Signal ./merged/signal/bbA_MA400_tanb60_2016.csv with 6600 events

Processing event 0
Signal ./merged/signal/bbH_MA1000_tanb60_2016.csv with 6712 events

Processing event 0
Signal ./merged/signal/bbH_MA500_tanb20_2016.csv with 6806 events

Processing event 0
Signal ./merged/signal/bbA_MA800_tanb30_2016.csv with 6964 events

Processing event 0
Signal ./merged/signal/bbH_MA200_tanb5_2016.csv with 1985 events

Processing event 0
Signal ./merged/signal/bbA_MA110_tanb45_2016.csv with 0 events
Signal ./merged/signal/bbA_MA225_tanb55_2016.csv with 0 events
Signal ./merge

Signal ./merged/signal/bbA_MA110_tanb25_2016.csv with 1514 events

Processing event 0
Signal ./merged/signal/bbA_MA180_tanb30_2016.csv with 1873 events

Processing event 0
Signal ./merged/signal/bbA_MA250_tanb35_2016.csv with 2092 events

Processing event 0
Signal ./merged/signal/bbA_MA120_tanb15_2016.csv with 1683 events

Processing event 0
Signal ./merged/signal/bbA_MA110_tanb10_2016.csv with 1572 events

Processing event 0
Signal ./merged/signal/bbH_MA130_tanb25_2016.csv with 1745 events

Processing event 0
Signal ./merged/signal/bbA_MA1000_tanb60_2016.csv with 6724 events

Processing event 0
Signal ./merged/signal/bbA_MA600_tanb25_2016.csv with 6951 events

Processing event 0
Signal ./merged/signal/bbA_MA170_tanb50_2016.csv with 0 events
Signal ./merged/signal/bbA_MA275_tanb45_2016.csv with 2120 events

Processing event 0
Signal ./merged/signal/bbA_MA140_tanb55_2016.csv with 0 events
Signal ./merged/signal/bbH_MA160_tanb60_2016.csv with 0 events
Signal ./merged/signal/bbH_MA300_tan

Signal ./merged/signal/bbH_MA700_tanb15_2016.csv with 7024 events

Processing event 0
Signal ./merged/signal/bbA_MA250_tanb45_2016.csv with 2100 events

Processing event 0
Signal ./merged/signal/bbA_MA180_tanb40_2016.csv with 0 events
Signal ./merged/signal/bbA_MA110_tanb60_2016.csv with 0 events
Signal ./merged/signal/bbH_MA130_tanb55_2016.csv with 0 events
Signal ./merged/signal/bbA_MA130_tanb20_2016.csv with 1715 events

Processing event 0
Signal ./merged/signal/bbH_MA110_tanb15_2016.csv with 1734 events

Processing event 0
Signal ./merged/signal/bbH_MA120_tanb10_2016.csv with 1730 events

Processing event 0
Signal ./merged/signal/bbH_MA250_tanb30_2016.csv with 2079 events

Processing event 0
Signal ./merged/signal/bbH_MA180_tanb35_2016.csv with 0 events
Signal ./merged/signal/bbA_MA700_tanb60_2016.csv with 6787 events

Processing event 0
Signal ./merged/signal/bbH_MA600_tanb20_2016.csv with 6984 events

Processing event 0
Signal ./merged/signal/bbA_MA300_tanb35_2016.csv with 6392 e

Signal ./merged/signal/bbH_MA130_tanb50_2016.csv with 0 events
Signal ./merged/signal/bbA_MA600_tanb50_2016.csv with 6854 events

Processing event 0
Signal ./merged/signal/bbH_MA700_tanb10_2016.csv with 7131 events

Processing event 0
Signal ./merged/signal/bbA_MA1000_tanb15_2016.csv with 7221 events

Processing event 0
Signal ./merged/signal/bbH_MA190_tanb5_2016.csv with 1911 events

Processing event 0
Signal ./merged/signal/bbH_MA700_tanb25_2016.csv with 7003 events

Processing event 0
Signal ./merged/signal/bbA_MA450_tanb30_2016.csv with 6710 events

Processing event 0
Signal ./merged/signal/bbA_MA1000_tanb20_2016.csv with 7089 events

Processing event 0
Signal ./merged/signal/bbA_MA130_tanb5_2016.csv with 1764 events

Processing event 0
Signal ./merged/signal/bbH_MA190_tanb40_2016.csv with 0 events
Signal ./merged/signal/bbH_MA350_tanb35_2016.csv with 6555 events

Processing event 0
Signal ./merged/signal/bbA_MA120_tanb55_2016.csv with 0 events
Signal ./merged/signal/bbA_MA225_tanb

Signal ./merged/signal/bbH_MA160_tanb40_2016.csv with 0 events
Signal ./merged/signal/bbH_MA150_tanb45_2016.csv with 0 events
Signal ./merged/signal/bbH_MA300_tanb10_2016.csv with 6433 events

Processing event 0
Signal ./merged/signal/bbA_MA450_tanb5_2016.csv with 6791 events

Processing event 0
Signal ./merged/signal/bbA_MA400_tanb15_2016.csv with 6704 events

Processing event 0
Signal ./merged/signal/bbH_MA1000_tanb15_2016.csv with 7092 events

Processing event 0
Signal ./merged/signal/bbA_MA800_tanb45_2016.csv with 6935 events

Processing event 0
Signal ./merged/signal/bbH_MA500_tanb55_2016.csv with 6782 events

Processing event 0
Signal ./merged/signal/bbH_MA600_tanb60_2016.csv with 6874 events

Processing event 0
Signal ./merged/signal/bbH_MA450_tanb35_2016.csv with 6631 events

Processing event 0
Signal ./merged/signal/bbA_MA700_tanb20_2016.csv with 7094 events

Processing event 0
Signal ./merged/signal/bbH_MA225_tanb45_2016.csv with 2058 events

Processing event 0
Signal ./merge

Signal ./merged/signal/bbA_MA500_tanb50_2016.csv with 6745 events

Processing event 0
Signal ./merged/signal/bbH_MA800_tanb40_2016.csv with 6909 events

Processing event 0
Signal ./merged/signal/bbH_MA400_tanb10_2016.csv with 6714 events

Processing event 0
Signal ./merged/signal/bbH_MA170_tanb5_2016.csv with 1918 events

Processing event 0
Signal ./merged/signal/bbH_MA400_tanb25_2016.csv with 6453 events

Processing event 0
Signal ./merged/signal/bbA_MA900_tanb35_2016.csv with 6987 events

Processing event 0
Signal ./merged/signal/bbH_MA200_tanb60_2016.csv with 0 events
Signal ./merged/signal/bbH_MA170_tanb40_2016.csv with 0 events
Signal ./merged/signal/bbH_MA140_tanb45_2016.csv with 0 events
Signal ./merged/signal/bbH_MA275_tanb55_2016.csv with 0 events
Signal ./merged/signal/bbA_MA300_tanb20_2016.csv with 6439 events

Processing event 0
Signal ./merged/signal/bbH_MA450_tanb60_2016.csv with 6668 events

Processing event 0
Signal ./merged/signal/bbH_MA600_tanb35_2016.csv with 6820 ev

In [ ]:
#Analysis on background. Scaling factors: luminosity, xSec, PU_Weight

for b in bkg_data:
    b_df = pd.read_csv(b[0])
    n = len(b_df)
    print('Background {} with {} events'.format(b[0], n))
    features = b_df[b_df.columns & nn_features].to_numpy()
    features = scaler.fit_transform(features)
    val = model.predict(features, batch_size = batch_size)
    for v in variables:
        plot[b[0]][v[0]] = TH1F(b[0]+v[1], v[2], int(v[3]), int(v[4]), int(v[5]))
    plot[b[0]]['invMass'] = TH1F(b[0]+'invariant_mass', 'dimuon invariant mass; Mass; entries;', 700, 0, 1400)
    i=0
    for index, row in b_df.iterrows():
        if i%10000 == 0:
            print('\nProcessing event {}'.format(i))
        if cat != 'cat2':
            bjet_n = row["bjet_n"]
        ljet_n = row["ljet_n"]
        
        M = row["dimuon_M"]
        PU_Weight = row["PU_Weight"]
        dimuon_deltar = row["dimuon_deltar"]
        dimuon_deltaphi = row["dimuon_deltaphi"]
        dimuon_deltaeta = row["dimuon_deltaeta"]
        met_pt = row["met_pt"]
        bkg_plot.Fill(val[i], M, PU_Weight)
        if val[i] > best_cut:
            scale =  PU_Weight
            plot[b[0]]['invMass'].Fill(M, scale)
            plot[b[0]]["dimuon_deltar"].Fill(dimuon_deltar, scale)
            plot[b[0]]["dimuon_deltaphi"].Fill(dimuon_deltaphi, scale)
            plot[b[0]]["dimuon_deltaeta"].Fill(dimuon_deltaeta, scale)
            plot[b[0]]["met_pt"].Fill(met_pt, scale)
            plot[b[0]]["ljet_n"].Fill(ljet_n, scale)
            
            if cat != 'cat2':
                plot[b[0]]["bjet_n"].Fill(bjet_n, scale)
                bjet_1_pt = row["bjet_1_pt"]
                bjet_1_eta = row["bjet_1_eta"]
                deltar_bjet1_dimuon = row["deltar_bjet1_dimuon"]
                deltapt_bjet1_dimuon = row["deltapt_bjet1_dimuon"]
                deltaeta_bjet1_dimuon = row["deltaeta_bjet1_dimuon"]
                
                if bjet_1_pt > 0:
                    plot[b[0]]["bjet_1_pt"].Fill(bjet_1_pt, scale)
                    plot[b[0]]["bjet_1_eta"].Fill(bjet_1_eta, scale)
                    if deltar_bjet1_dimuon > 0:
                        plot[b[0]]["deltar_bjet1_dimuon"].Fill(deltar_bjet1_dimuon, scale)
                    if deltapt_bjet1_dimuon > 0:
                        plot[b[0]]["deltapt_bjet1_dimuon"].Fill(deltapt_bjet1_dimuon, scale)
                    if deltaeta_bjet1_dimuon > 0:
                        plot[b[0]]["deltaeta_bjet1_dimuon"].Fill(deltaeta_bjet1_dimuon, scale)
            
            ljet_1_pt = row["ljet_1_pt"]
            ljet_1_eta = row["ljet_1_eta"]
            if ljet_1_pt > 0:
                plot[b[0]]["ljet_1_pt"].Fill(ljet_1_pt, scale)
                plot[b[0]]["ljet_1_eta"].Fill(ljet_1_eta, scale)
        i=i+1
        
    print("\n >>> Finisced with {} file...\n".format(b[0]))

print("\n>>>Single plots are done. Starting merge...<<<\n")

#Adding GetEntries/Integral to scaling factors
t = 1
for b in bkg_data:

    leg_bool=False
    for group in groups:
        if t==group:
            leg_bool=True

    scale =  plot[b[0]]['invMass'].GetEntries()/plot[b[0]]['invMass'].Integral() if ( plot[b[0]]['invMass'].Integral()> 0) else 1
    b_inv_mass.Add(plot[b[0]]['invMass'], lumi * b[1] * scale)
    for v in variables:
        scale =  plot[b[0]][v[0]].GetEntries()/plot[b[0]][v[0]].Integral() if ( plot[b[0]][v[0]].Integral()>  0) else 1
        #b_plots[v[0]].Add(plot[b[0]][v[0]], lumi * b[1] * scale)
        _post = plot[b[0]][v[0]]
        _post.SetFillColor(b[2])
        if b[2] == 1:
            tmpsig0[v[0]].Add(_post, lumi * b[1] * scale)
            if leg_bool == True:
                tmpsig0[v[0]].SetLineColor(1)
                tmpsig0[v[0]].SetLineStyle(1)
                tmpsig0[v[0]].SetLineWidth(3)
                tmpsig0[v[0]].Scale(0.001)
                legend_stack[v[0]].AddEntry(_post,"A m_{A}="+ str(massA) + "GeV x 100","le")

        if b[2] == 3:
            tmpbkg0[v[0]].Add(_post, lumi* b[1] *scale)
            if leg_bool==True:
                tmpbkg0[v[0]].SetLineColor(1)
                tmpbkg0[v[0]].SetFillColor(b[2])
                stacked_histo[v[0]].Add(tmpbkg0[v[0]])
                legend_stack[v[0]].AddEntry(_post,"Drell-Yan","F")
        if b[2] == 2:
            tmpbkg1[v[0]].Add(_post, lumi* b[1] *scale)
            if leg_bool==True:
                tmpbkg1[v[0]].SetLineColor(1)
                tmpbkg1[v[0]].SetFillColor(b[2])
                stacked_histo[v[0]].Add(tmpbkg1[v[0]])
                legend_stack[v[0]].AddEntry(_post,"t#bar{t}","F")
        if b[2] == 4:
            tmpbkg2[v[0]].Add(_post, lumi* b[1] *scale)
            if leg_bool==True:
                tmpbkg2[v[0]].SetLineColor(1)
                tmpbkg2[v[0]].SetFillColor(b[2])
                stacked_histo[v[0]].Add(tmpbkg2[v[0]])
                legend_stack[v[0]].AddEntry(_post,"Single t","F")

        if b[2] == 5:
            tmpbkg3[v[0]].Add(_post, lumi* b[1] *scale)
            if leg_bool==True:
                tmpbkg3[v[0]].SetLineColor(1)
                tmpbkg3[v[0]].SetFillColor(b[2])
                stacked_histo[v[0]].Add(tmpbkg3[v[0]])
                legend_stack[v[0]].AddEntry(_post,"W^{+}W^{-}, W^{#pm}Z, ZZ","F")
        if b[2]!=1:
                b_plots[v[0]].Add(_post, lumi* b[1] *scale)

    t=t+1

In [ ]:
for d in real_data:
    d_df = pd.read_csv(d[0])
    n = len(d_df)
    print('Data {} with {} events'.format(d[0], n))
    features = d_df[d_df.columns & nn_features].to_numpy()
    features = scaler.fit_transform(features)
    val = model.predict(features, batch_size = batch_size)
    
    i=0
    for index, row in d_df.iterrows():
        if i%10000 == 0:
            print('\nProcessing event {}'.format(i))

        if val[i] > best_cut:
            PU_Weight = row["PU_Weight"]
            M = row["dimuon_M"]
            dimuon_deltar = row["dimuon_deltar"]
            dimuon_deltaphi = row["dimuon_deltaphi"]
            dimuon_deltaeta = row["dimuon_deltaeta"]
            met_pt = row["met_pt"]
            ljet_n = row["ljet_n"]
            scale = PU_Weight *d[1]
            d_inv_mass.Fill(M, scale)
            d_plots["dimuon_deltar"].Fill(dimuon_deltar, scale)
            d_plots["dimuon_deltaphi"].Fill(dimuon_deltaphi, scale)
            d_plots["dimuon_deltaeta"].Fill(dimuon_deltaeta, scale)
            d_plots["met_pt"].Fill(met_pt, scale)
            d_plots["ljet_n"].Fill(ljet_n, scale)
            if cat != 'cat2':
                bjet_n = row["bjet_n"]
                bjet_1_pt = row["bjet_1_pt"]
                bjet_1_eta = row["bjet_1_eta"]
                deltar_bjet1_dimuon = row["deltar_bjet1_dimuon"]
                deltapt_bjet1_dimuon = row["deltapt_bjet1_dimuon"]
                deltaeta_bjet1_dimuon = row["deltaeta_bjet1_dimuon"]
                d_plots["bjet_n"].Fill(bjet_n, scale)
                if bjet_1_pt > 0:
                    d_plots["bjet_1_pt"].Fill(bjet_1_pt, scale)
                    d_plots["bjet_1_eta"].Fill(bjet_1_eta, scale)
                    if deltar_bjet1_dimuon > 0:
                        d_plots["deltar_bjet1_dimuon"].Fill(deltar_bjet1_dimuon, scale)
                    if deltapt_bjet1_dimuon > 0:
                        d_plots["deltapt_bjet1_dimuon"].Fill(deltapt_bjet1_dimuon, scale)
                    if deltaeta_bjet1_dimuon > 0:
                        d_plots["deltaeta_bjet1_dimuon"].Fill(deltaeta_bjet1_dimuon, scale)
            ljet_1_pt = row["ljet_1_pt"]
            ljet_1_eta = row["ljet_1_eta"]
            if ljet_1_pt > 0:
                d_plots["ljet_1_pt"].Fill(ljet_1_pt, scale)
                d_plots["ljet_1_eta"].Fill(ljet_1_eta, scale)
        i = i+1

In [ ]:
pad1 = {}
pad2 = {}
ratios = {}
canvas = {}
legend = {}

for v in variables:
    canvas[v[0]] = TCanvas(v[0], v[0])

    pad1[v[0]] = TPad("pad1", "pad1", 0, 0.3, 1, 1.0)
    pad1[v[0]].SetBottomMargin(0)
    pad1[v[0]].SetGridx()
    pad1[v[0]].SetGridy()
    pad1[v[0]].SetLogy(1)
    pad1[v[0]].Draw()
    pad1[v[0]].cd()

    b_plots[v[0]].SetStats(0)
    stacked_histo[v[0]].Draw("hist F")
    tmpsig0[v[0]].Draw("same")
    #b_plots[v[0]].SetFillColor(4)
    #b_plots[v[0]].Draw("hist F")
    d_plots[v[0]].SetMarkerStyle(8)
    d_plots[v[0]].Draw("SAME E")

    #legend[v[0]] = TLegend(0.1,0.7,0.48,0.9)
    #legend[v[0]].SetHeader("legend", "C")
    #legend[v[0]].AddEntry(b_plots[v[0]], "Monte Carlo")
    #legend[v[0]].AddEntry(d_plots[v[0]], "Data")
    #legend[v[0]].Draw()
    legend_stack[v[0]].AddEntry(d_plots[v[0]], "Data")
    legend_stack[v[0]].SetBorderSize(0)
    legend_stack[v[0]].SetLineWidth(0)
    legend_stack[v[0]].SetFillColor(0)
    legend_stack[v[0]].Draw()


    canvas[v[0]].cd()
    pad2[v[0]] = TPad("pad2", "pad2", 0, 0.05, 1, 0.3)
    pad2[v[0]].SetTopMargin(0.2)
    pad2[v[0]].SetGridx()
    pad2[v[0]].SetGridy()
    pad2[v[0]].Draw()
    pad2[v[0]].cd()

    ratios[v[0]] = d_plots[v[0]].Clone("ratio")
    ratios[v[0]].GetYaxis().SetTitle("Data/MC")
    ratios[v[0]].SetLineColor(1)
    ratios[v[0]].SetMinimum(0)
    ratios[v[0]].SetMaximum(2)
    ratios[v[0]].Sumw2()
    ratios[v[0]].SetStats(0)
    ratios[v[0]].Divide(b_plots[v[0]])
    ratios[v[0]].SetMarkerStyle(8)
    ratios[v[0]].Draw("ep")

canvas['invMass'] = TCanvas('invMass', 'invMass')
pad1['invMass'] = TPad("pad1", "pad1", 0, 0.3, 1, 1.0)
pad1['invMass'].SetBottomMargin(0)
pad1['invMass'].SetGridx()
pad1['invMass'].SetGridy()
pad1['invMass'].SetLogy(1)
pad1['invMass'].Draw()
pad1['invMass'].cd()

b_inv_mass.SetStats(0)
b_inv_mass.SetFillColor(4)
b_inv_mass.Draw("hist F")
d_inv_mass.SetMarkerStyle(8)
d_inv_mass.Draw("SAME E")
s_inv_mass.SetMarkerStyle(34)
s_inv_mass.Draw("SAME E")

legend['invMass'] = TLegend(0.1,0.7,0.48,0.9)
legend['invMass'].SetHeader("legend", "C")
legend['invMass'].AddEntry(b_inv_mass, "Monte Carlo")
legend['invMass'].AddEntry(d_inv_mass, "Data")
legend['invMass'].AddEntry(s_inv_mass, "bb Signal")
legend['invMass'].Draw()

canvas['invMass'].cd()
pad2['invMass'] = TPad("pad2", "pad2", 0, 0.05, 1, 0.3)
pad2['invMass'].SetTopMargin(0.2)
pad2['invMass'].SetGridx()
pad2['invMass'].SetGridy()
pad2['invMass'].Draw()
pad2['invMass'].cd()

ratios['invMass'] = d_inv_mass.Clone("ratio")
ratios['invMass'].GetYaxis().SetTitle("Data/MC")
ratios['invMass'].SetLineColor(1)
ratios['invMass'].SetMinimum(0)
ratios['invMass'].SetMaximum(2)
ratios['invMass'].Sumw2()
ratios['invMass'].SetStats(0)
ratios['invMass'].Divide(b_inv_mass)
ratios['invMass'].SetMarkerStyle(8)
ratios['invMass'].Draw("ep")

s_inv_mass.Scale(NORMALIZZAZIONE)

Ns = s_inv_mass.Integral(s_inv_mass.FindBin(range_down), s_inv_mass.FindBin(range_up))
Nb = b_inv_mass.Integral(b_inv_mass.FindBin(range_down), b_inv_mass.FindBin(range_up))
significance_MC_signal = (Ns/Sqrt(Ns+Nb), Ns/Sqrt(Nb))


string3 = "\nLa massa vale {}, Ns vale {} e Nb vale {}\n".format(massA, Ns, Nb)
string4 = "range_down {} corrispinde a {} e range_up {} a {}".format(range_down, s_inv_mass.FindBin(range_down), range_up, s_inv_mass.FindBin(range_up))
string1 ='\nSignificance MC at {}+-10% GeV is(s/sqrt(s+b)): {}\n'.format(massA, significance_MC_signal[0])
string2 = '\nSignificance MC at {}+-10% GeV is(s/sqrt(b)): {}\n'.format(massA, significance_MC_signal[1])
print(string3)
print(string4)
print(string1)
print(string2)
#with open('significance_Keras{}.txt'.format(cat), 'w+') as f:
with open('significancePNN_{}.txt'.format(cat), 'w+') as f:
    f.write(string3)
    f.write(string4)
    f.write(string1)
    f.write(string2)

    f.close()

for v in variables:
    output_file.WriteTObject(s_plots[v[0]])
    output_file.WriteTObject(canvas[v[0]])
output_file.WriteTObject(sgn_plot)
output_file.WriteTObject(bkg_plot)
output_file.WriteTObject(canvas['invMass'])
output_file.WriteTObject(s_inv_mass)
output_file.WriteTObject(b_inv_mass)
output_file.WriteTObject(d_inv_mass)

output_file.Close()